<a href="https://colab.research.google.com/github/daniellaeme/Fake-News-Detection/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Importing all Libraries

In [ ]:
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
import re
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier

from sklearn.metrics import accuracy_score, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Data Pre-processing

In [ ]:
import pandas as pd

In [ ]:
fake = pd.read_csv('/content/gdrive/MyDrive/datasets/Copy of Fake.csv')
real = pd.read_csv('/content/gdrive/MyDrive/datasets/Copy of True.csv')



In [ ]:
fake.head(3)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"


In [ ]:
real.head(3)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"


In [ ]:
fake.shape, real.shape

((23481, 4), (21417, 4))

In [ ]:
fake.isna().any().sum(), real.isna().any().sum()

(0, 0)

There are no missing values in this dataset

In [ ]:
fake['label'] = 0
real['label'] = 1

df = pd.concat([fake, real])
df.reset_index(drop=True, inplace=True)
df.shape

(44898, 5)

In [ ]:
data = df.copy()

# Data Pre-processing

In [ ]:
data['full_text'] = data['title'] + data['text']
data.head(2)

,title,text,subject,date,label,full_text
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0,Donald Trump Sends Out Embarrassing New Year’...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0,Drunk Bragging Trump Staffer Started Russian ...


In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()

import re
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')

from tqdm import tqdm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
stop_words = set(stopwords.words('english'))
def clean_text(sentence):
    updated_words = []
    ps = PorterStemmer()
    word_tokens = word_tokenize(sentence)
    for word in word_tokens:
        if word not in stop_words:
            word = re.sub('[^a-zA-Z0-9]', '',word)
            word = word.lower()
            word = ps.stem(word)
            updated_words.append(word)
    return " ".join(updated_words)

In [ ]:
tqdm.pandas()

In [ ]:
data['full_text'] = data['full_text'].progress_apply(clean_text)

100%|██████████| 44898/44898 [11:25<00:00, 65.52it/s]


In [ ]:
X = data['full_text']
y = data['label']

## Data Splitting

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=1)

## Tfidf Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vector = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))
X_train = vector.fit_transform(X_train)
X_test = vector.transform(X_test)

# Model Building

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
lr = LogisticRegression(random_state=1)
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

print('Logistic Regression')
print('Accuracy:', accuracy_score(y_test, lr_pred))
print('Classification Report:', classification_report(y_test, lr_pred))

Logistic Regression
Accuracy: 0.9920935412026726
Classification Report:               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4682
           1       0.99      0.99      0.99      4298

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier(random_state=1)
dtc.fit(X_train, y_train)
dtc_pred = dtc.predict(X_test)

print('DecisionTreeClassifier')
print('Accuracy:', accuracy_score(y_test, dtc_pred))
print('Classification Report:', classification_report(y_test, dtc_pred))

DecisionTreeClassifier
Accuracy: 0.9955456570155902
Classification Report:               precision    recall  f1-score   support

           0       0.99      1.00      1.00      4682
           1       1.00      0.99      1.00      4298

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [ ]:
pac = PassiveAggressiveClassifier(random_state=1)
pac.fit(X_train, y_train)
pac_pred = pac.predict(X_test)

print('PassiveAggressiveClasifier')
print('Accuracy:', accuracy_score(y_test, pac_pred))
print('Classification Report:', classification_report(y_test, pac_pred))

PassiveAggressiveClasifier
Accuracy: 0.9953229398663697
Classification Report:               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4682
           1       0.99      1.00      1.00      4298

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



Hence, the Passive Aggressive Classifier is the best model as it perfectly detects fake news.

In [ ]:
lr.predict(X_test[1234])

array([0])

In [ ]:
dtc.predict(X_test[1234])

array([0])

In [ ]:
pac.predict(X_test[1234])

array([0])

# Save Model

In [ ]:
import pickle

In [ ]:
pickle.dump(lr, open('model1.pkl', 'wb'))

In [ ]:
pickle.dump(dtc, open('model2.pkl', 'wb'))

In [ ]:
pickle.dump(pac, open('model3.pkl', 'wb'))

In [ ]:
pickle.dump(vector, open('tfidfvect', 'wb'))

In [ ]:
# @title
import csv

# Open the CSV file
with open("/content/gdrive/MyDrive/datasets/Copy of True.csv", "r") as f:
    reader = csv.reader(f)

    # Iterate over the rows in the CSV file
    for row in reader:

        # Check if the word "https" is in the row
        if "https" in row:

            # Do something if the word "https" is found
            print("https found in row:", row)


In [ ]:
# @title
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('punkt')
import nltk
nltk.download('wordnet')
import nltk
nltk.download('stopwords')

# PorterStemmer
def clean_text_1(text):
    porter_stemmer = PorterStemmer()
    return porter_stemmer.stem(text)

# NLTK word_tokenize() and stem()
def clean_text_2(text):
    words = nltk.tokenize.word_tokenize(text)
    stemmed_words = [nltk.stem.WordNetLemmatizer().lemmatize(word) for word in words]
    return " ".join(stemmed_words)

# re
def clean_text_3(text):
    text = re.sub(r"[^\w\s]", "", text)
    text = text.lower()
    words = text.split()
    stop_words = set(stopwords.words("english"))
    stemmed_words = [nltk.stem.WordNetLemmatizer().lemmatize(word) for word in words if word not in stop_words]
    return " ".join(stemmed_words)

# Example usage
text = "This is a sample text with some stop words, such as the, is, and a."

# PorterStemmer
cleaned_text_1 = clean_text_1(text)
print(cleaned_text_1)

# NLTK word_tokenize() and stem()
cleaned_text_2 = clean_text_2(text)
print(cleaned_text_2)

# re
cleaned_text_3 = clean_text_3(text)
print(cleaned_text_3)
